<a href="https://colab.research.google.com/github/Yazeedx0/ProgresSoft/blob/main/LLM_NLP_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Download Dataset from kaggle**

In [7]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.23GB/s]


In [9]:
!unzip /content/imdb-dataset-of-50k-movie-reviews.zip -d /content/dataset

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: /content/dataset/IMDB Dataset.csv  


## **Text Processing**


In [17]:
import pandas as pd
dataset = pd.read_csv('/content/dataset/IMDB Dataset.csv')
print(dataset.head)
print(dataset.columns)

<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>
Index(['review', 'sentiment'], dtype='object')


In [18]:
!pip install pandas nltk -qU

In [19]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle
from google.colab import drive
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
stop_words = set(stopwords.words('english')) - {'not', 'very', 'too', 'quite'}
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
  text = text.lower()

  text = re.sub(r'<.*?>', '', text)

  text = re.sub(r'\S+@\S+\.\S+', '', text)
  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.IGNORECASE)
  text = re.sub(r'[^\w\s]', '', text)

  words = text.split()
  words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
  text = ' '.join(words)
  return text

dataset['processed_review'] = dataset['review'].apply(preprocess_text)

print(dataset[['review', 'processed_review']].head())

                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                    processed_review  
0  one reviewer mentioned watching 1 oz episode y...  
1  wonderful little production filming technique ...  
2  thought wonderful way spend time too hot summe...  
3  basically there family little boy jake think t...  
4  petter matteis love time money visually stunni...  


In [21]:
with open('/content/dataset/processed_dataset.pkl', 'wb') as f:
  pickle.dump(dataset[['processed_review', 'sentiment']], f)

drive.mount('/content/drive')
!cp /content/dataset/processed_dataset.pkl /content/drive/MyDrive/processed_dataset.pkl

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Machine Learning Model for Sentiment Classification**

In [2]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from google.colab import drive

In [11]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['processed_review'])
y = data['sentiment'].map({'positive': 1, 'negative': 0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [16]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
fpr = fp / (fp + tn)
fnr = fn / (fn + tp)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print output
print(f"Accuracy: {accuracy:.4f}")
print(f"False Positive Rate (FPR): {fpr:.4f}")
print(f"False Negative Rate (FNR): {fnr:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.8871
False Positive Rate (FPR): 0.1268
False Negative Rate (FNR): 0.0992
Precision: 0.8783
Recall: 0.9008
F1-Score: 0.8894


In [20]:
with open ('/content/dataset/model.pkl', 'wb') as f:
  pickle.dump(model, f)

with open('/content/dataset/vectorizer.pkl', 'wb') as f:
  pickle.dump(vectorizer, f)

!cp /content/dataset/model.pkl /content/drive/MyDrive/lr_model.pkl
!cp /content/dataset/vectorizer.pkl /content/drive/MyDrive/tfidf_vectorizer.pkl

## **LSTM Model for Sentiment Classification**

In [ ]:
!pip install tensorflow

In [30]:
import pandas as pd
import pickle
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from google.colab import drive

In [31]:
drive.mount('/content/drive')

with open('/content/drive/MyDrive/processed_dataset.pkl', 'rb') as f:
    processed_data = pickle.load(f)

raw_data = pd.read_csv('/content/dataset/IMDB Dataset.csv')

print("Processed Data:")
print(processed_data.head())
print("\nRaw Data:")
print(raw_data.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed Data:
                                    processed_review sentiment
0  one reviewer mentioned watching 1 oz episode y...  positive
1  wonderful little production filming technique ...  positive
2  thought wonderful way spend time too hot summe...  positive
3  basically there family little boy jake think t...  negative
4  petter matteis love time money visually stunni...  positive

Raw Data:
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [33]:
max_words = 7000
mex_len = 250

def prepare_text_data(texts, tokenizer=None, fit = True):
  if fit:
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts)
  sequences = tokenizer.texts_to_sequences(texts)
  data = pad_sequences(sequences, maxlen=mex_len)
  return data, tokenizer

X_processed, tokenizer_processed = prepare_text_data(processed_data['processed_review'], fit=True)
y_processed = processed_data['sentiment'].map({'positive': 1, 'negative': 0})
X_raw, tokenizer_raw = prepare_text_data(raw_data['review'], fit=True)
y_raw = raw_data['sentiment'].map({'positive': 1, 'negative': 0})

X_train_proc, X_test_proc, y_train_proc, y_test_proc = train_test_split(X_processed, y_processed, test_size=0.2, random_state=42)
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42)

In [35]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d /content/glove

--2025-08-02 10:24:44--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-08-02 10:24:44--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-08-02 10:24:45--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [37]:
embedding_dim = 100
embeddings_index = {}
with open('/content/glove/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer_processed.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [38]:
max_words = 7000
max_len = 250

def build_lstm_model(vocab_size, max_len, embedding_matrix=None):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
    else:
        model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.6))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
model_processed = build_lstm_model(max_words, max_len)
print("Training LSTM on processed data...")
model_processed.fit(X_train_proc, y_train_proc, epochs=5, batch_size=64, validation_split=0.2, verbose=1)

Training LSTM on processed data...
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.7562 - loss: 0.4802 - val_accuracy: 0.8774 - val_loss: 0.2920
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9144 - loss: 0.2323 - val_accuracy: 0.8831 - val_loss: 0.2852
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9360 - loss: 0.1760 - val_accuracy: 0.8779 - val_loss: 0.3495
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9512 - loss: 0.1367 - val_accuracy: 0.8710 - val_loss: 0.3419
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9635 - loss: 0.1062 - val_accuracy: 0.8681 - val_loss: 0.3856


In [39]:
model_processed = build_lstm_model(max_words, max_len, embedding_matrix)
print("Training LSTM on processed data...")
model_processed.fit(X_train_proc, y_train_proc, epochs=7, batch_size=64, validation_split=0.2, verbose=1)

Training LSTM on processed data...
Epoch 1/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.7014 - loss: 0.5639 - val_accuracy: 0.8256 - val_loss: 0.3952
Epoch 2/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.8247 - loss: 0.4021 - val_accuracy: 0.8342 - val_loss: 0.3763
Epoch 3/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.8443 - loss: 0.3629 - val_accuracy: 0.8668 - val_loss: 0.3274
Epoch 4/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.8618 - loss: 0.3325 - val_accuracy: 0.8705 - val_loss: 0.3112
Epoch 5/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.8749 - loss: 0.3094 - val_accuracy: 0.8746 - val_loss: 0.3022
Epoch 6/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8799 - loss: 0.2939 - val_accuracy: 0.8806 - val_loss: 0.2929
Epoch 7/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.8812 - loss: 0.2870 - val_accuracy: 0.8804 - val_loss: 0.2909


In [40]:
model_raw = build_lstm_model(max_words, max_len)
print("Training LSTM on raw data...")
model_raw.fit(X_train_raw, y_train_raw, epochs=7, batch_size=64, validation_split=0.2, verbose=1)

Training LSTM on raw data...
Epoch 1/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.7344 - loss: 0.5099 - val_accuracy: 0.8677 - val_loss: 0.3068
Epoch 2/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.8847 - loss: 0.2853 - val_accuracy: 0.8739 - val_loss: 0.3047
Epoch 3/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9215 - loss: 0.2105 - val_accuracy: 0.8826 - val_loss: 0.2893
Epoch 4/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9422 - loss: 0.1578 - val_accuracy: 0.8745 - val_loss: 0.3152
Epoch 5/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9491 - loss: 0.1416 - val_accuracy: 0.8805 - val_loss: 0.3268
Epoch 6/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.9620 - loss: 0.1052 - val_accuracy: 0.8792 - val_loss: 0.4000
Epoch 7/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.9675 - loss: 0.0930 - val_accuracy: 0.8767 - val_loss: 0.4294


In [41]:
y_pred_proc = (model_processed.predict(X_test_proc) > 0.5).astype("int32")
accuracy_proc = accuracy_score(y_test_proc, y_pred_proc)
cm_proc = confusion_matrix(y_test_proc, y_pred_proc)
tn, fp, fn, tp = cm_proc.ravel()
fpr_proc = fp / (fp + tn)
fnr_proc = fn / (fn + tp)
precision_proc = precision_score(y_test_proc, y_pred_proc)
recall_proc = recall_score(y_test_proc, y_pred_proc)
f1_proc = f1_score(y_test_proc, y_pred_proc)

print("Processed Data Metrics:")
print(f"Accuracy: {accuracy_proc:.4f}")
print(f"False Positive Rate (FPR): {fpr_proc:.4f}")
print(f"False Negative Rate (FNR): {fnr_proc:.4f}")
print(f"Precision: {precision_proc:.4f}")
print(f"Recall: {recall_proc:.4f}")
print(f"F1-Score: {f1_proc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Processed Data Metrics:
Accuracy: 0.8748
False Positive Rate (FPR): 0.1246
False Negative Rate (FNR): 0.1258
Precision: 0.8770
Recall: 0.8742
F1-Score: 0.8756


In [42]:
y_pred_raw = (model_raw.predict(X_test_raw) > 0.5).astype("int32")
accuracy_raw = accuracy_score(y_test_raw, y_pred_raw)
cm_raw = confusion_matrix(y_test_raw, y_pred_raw)
tn, fp, fn, tp = cm_raw.ravel()
fpr_raw = fp / (fp + tn)
fnr_raw = fn / (fn + tp)
precision_raw = precision_score(y_test_raw, y_pred_raw)
recall_raw = recall_score(y_test_raw, y_pred_raw)
f1_raw = f1_score(y_test_raw, y_pred_raw)

print("\nRaw Data Metrics:")
print(f"Accuracy: {accuracy_raw:.4f}")
print(f"False Positive Rate (FPR): {fpr_raw:.4f}")
print(f"False Negative Rate (FNR): {fnr_raw:.4f}")
print(f"Precision: {precision_raw:.4f}")
print(f"Recall: {recall_raw:.4f}")
print(f"F1-Score: {f1_raw:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step

Raw Data Metrics:
Accuracy: 0.8768
False Positive Rate (FPR): 0.1631
False Negative Rate (FNR): 0.0839
Precision: 0.8509
Recall: 0.9161
F1-Score: 0.8823


In [43]:
model_processed.save('/content/lstm_processed_model.h5')
model_raw.save('/content/lstm_raw_model.h5')
with open('/content/tokenizer_processed.pkl', 'wb') as f:
    pickle.dump(tokenizer_processed, f)
with open('/content/tokenizer_raw.pkl', 'wb') as f:
    pickle.dump(tokenizer_raw, f)
!cp /content/lstm_processed_model.h5 /content/drive/MyDrive/lstm_processed_model.h5
!cp /content/lstm_raw_model.h5 /content/drive/MyDrive/lstm_raw_model.h5
!cp /content/tokenizer_processed.pkl /content/drive/MyDrive/tokenizer_processed.pkl
!cp /content/tokenizer_raw.pkl /content/drive/MyDrive/tokenizer_raw.pkl

# **GPT-2 Classification using Prompt Engineering**

In [45]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from google.colab import drive
import re

In [46]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [49]:
example_positive = "This movie was fantastic! The acting was great and the story was very engaging."
example_negative = "I didn't like this movie. The plot was boring and the characters were uninteresting."

# Prompts لكل نوع
prompt_1shot = f"""Classify the following movie review as Positive or Negative:

Example:
Review: {example_positive}
Sentiment: Positive

Review: {{review}}
Sentiment: """

prompt_2shot = f"""Classify the following movie review as Positive or Negative:

Example 1:
Review: {example_positive}
Sentiment: Positive

Example 2:
Review: {example_negative}
Sentiment: Negative

Review: {{review}}
Sentiment: """

prompt_3shot = f"""Classify the following movie review as Positive or Negative:

Example 1:
Review: {example_positive}
Sentiment: Positive

Example 2:
Review: {example_negative}
Sentiment: Negative

Example 3:
Review: This film was a masterpiece with stunning visuals and a gripping plot.
Sentiment: Positive

Review: {{review}}
Sentiment: """

In [55]:
def classify_review(review, prompt_type="1shot"):
    if prompt_type == "1shot":
        prompt = prompt_1shot.format(review=review)
    elif prompt_type == "2shot":
        prompt = prompt_2shot.format(review=review)
    elif prompt_type == "3shot":
        prompt = prompt_3shot.format(review=review)
    else:
        raise ValueError("Prompt type must be '1shot', '2shot', or '3shot'")
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10, pad_token_id=tokenizer.eos_token_id)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    sentiment = generated_text.split("Sentiment:")[-1].strip()

    sentiment = sentiment.lower()
    if "positive" in sentiment:
        return "Positive"
    elif "negative" in sentiment:
        return "Negative"
    else:
        positive_words = ["good", "great", "fantastic", "amazing", "love"]
        negative_words = ["bad", "poor", "terrible", "boring", "hate"]
        review_lower = review.lower()
        pos_count = sum(review_lower.count(word) for word in positive_words)
        neg_count = sum(review_lower.count(word) for word in negative_words)
        return "Positive" if pos_count > neg_count else "Negative"

test_review = "This movie was amazing, I loved the plot!"
print(f"1-shot: {classify_review(test_review, '1shot')}")
print(f"2-shot: {classify_review(test_review, '2shot')}")
print(f"3-shot: {classify_review(test_review, '3shot')}")

1-shot: Positive
2-shot: Negative
3-shot: Negative


In [57]:
raw_data = pd.read_csv('/content/dataset/IMDB Dataset.csv')
print(raw_data.head())

sample_data = raw_data.sample(1000, random_state=42)

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [58]:
def evaluate_prompt(prompt_type, data):
    y_true = data['sentiment'].map({'positive': 'Positive', 'negative': 'Negative'}).tolist()
    y_pred = [classify_review(review, prompt_type) for review in data['review']]

    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=["Positive", "Negative"])
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    precision = precision_score(y_true, y_pred, pos_label="Positive")
    recall = recall_score(y_true, y_pred, pos_label="Positive")
    f1 = f1_score(y_true, y_pred, pos_label="Positive")

    return {
        "Accuracy": accuracy,
        "FPR": fpr,
        "FNR": fnr,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }

results_1shot = evaluate_prompt("1shot", sample_data)
results_2shot = evaluate_prompt("2shot", sample_data)
results_3shot = evaluate_prompt("3shot", sample_data)

print("1-shot Prompt Results:")
for metric, value in results_1shot.items():
    print(f"{metric}: {value:.4f}")

print("\n2-shot Prompt Results:")
for metric, value in results_2shot.items():
    print(f"{metric}: {value:.4f}")

print("\n3-shot Prompt Results:")
for metric, value in results_3shot.items():
    print(f"{metric}: {value:.4f}")

1-shot Prompt Results:
Accuracy: 0.5610
FPR: 0.6660
FNR: 0.2328
Precision: 0.5658
Recall: 0.3340
F1-Score: 0.4201

2-shot Prompt Results:
Accuracy: 0.5360
FPR: 0.9517
FNR: 0.0210
Precision: 0.6765
Recall: 0.0483
F1-Score: 0.0902

3-shot Prompt Results:
Accuracy: 0.5270
FPR: 0.7500
FNR: 0.2214
Precision: 0.5064
Recall: 0.2500
F1-Score: 0.3347
